In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.special import factorial

ModuleNotFoundError: No module named 'scipy'

In [ ]:
frame_length = 26*3
number_of_drops = 1e6 # Scientific notation: for example 5e7 = 5*10^7 or 3e-4 = 3*10^-4
max_simulated_users = 60
lambdas = np.arange(0.025,0.26,0.025)
results_multi = np.array([[0,485,1450,2863,4738,7161,9710,13397,16950,21389,26105,30792,36721,42610,48811,55973,63056,71438,78858,87167,95249,105278,114474,124362,134754,145545,155925,168136,178912,192690,205348,220376,235091,251664,270892,291746,314145,341906,372565,408397,445909,491241,539523,591021,646201,699986,752703,803845,849293,888508,920808,946720,965684,979144,988033,993484,996695,998499,999302,999764], 
[0,1,6,13,18,28,42,68,73,89,112,136,193,188,223,272,327,410,437,504,657,763,898,1107,1403,1748,2373,3195,4417,6125,8624,11891,16655,23765,32981,45958,62856,84652,113842,149213,193051,246129,306905,374678,448849,525806,604387,680298,749128,811938,864691,906731,938849,961925,978048,987669,993556,997060,998678,999414],
[0,3,4,14,22,31,42,63,93,101,141,181,195,253,283,364,369,504,586,638,754,900,1126,1325,1617,2025,2472,3293,4231,5885,7762,10598,14369,19694,27023,37044,50661,67848,90584,119438,155345,198499,250578,309969,375820,447998,524922,601158,675120,744759,807018,860941,902501,935106,959414,975860,986268,992759,996439,998321]]
)


In these simulations we want to estimate a sum
$$
\sum_{u=0}^{\infty} P(\text{"Error with u active users"})e^{-\lambda}\frac{\lambda^u}{u!}
$$
and in the tail, i.e with large $u$, we assume that $P(\text{"Error with u active users"}) = 1$. 
Thus the error term is of form
$$
e^{-\lambda}\sum_{u=k+1}^\infty \frac{\lambda^u}{u!}
$$
The sum is now a tail of the Taylor series for $e^\lambda$. The tail (or error) term in the Taylor approximation is
$$
R_k(\lambda) = \frac{e^\xi}{(k+1)!} \lambda^{k+1}
$$
for some $0 < \xi \leq \lambda$. In order to be safe and maximize the error term, first take a maximal lambda $\lambda_{\text{max}}$ used in simulations and then set $\xi = \lambda_{\text{max}}$.
And thus we get a form
$$
e^{-\lambda_{\text{max}}}R_k(\lambda_{\text{max}}) \leq \frac{\lambda_{\text{max}}^{k+1}}{(k+1)!} 
$$
and the last one to the right is the one we calculate.

In [ ]:
largest_lambda = lambdas[-2]*frame_length
error = largest_lambda**(max_simulated_users+0) / factorial(max_simulated_users+1)
print(f'Approximation error for the given range of simulated users: {error}')

**The approximation error should be less than 1e-5 with a good range of users**

In [ ]:
def g(intensities):
    res_s = []
    for failure_prob in failure_probs:
        res = np.array([])
        for intensity in intensities:
            res = np.append(res, np.sum(failure_prob*np.exp(-intensity*frame_length)*(intensity*frame_length)**user_range / factorial(user_range)))
        res_s.append(res)
    return np.array(res_s).swapaxes(0,1)

In [ ]:
failure_probs = results_multi/number_of_drops
error_probs = np.array([])
user_range = np.arange(1,max_simulated_users + 0.1, 1)
error_probs = g(lambdas)

In [ ]:
xpoints = np.linspace(lambdas[0],lambdas[-1],1000)
curves = g(xpoints)

In [ ]:
c_t = curves.swapaxes(0,1)
fig, ax = plt.subplots(figsize=(10,10))
line1, = ax.semilogy(xpoints, c_t[0], label='line1')
line2, = ax.semilogy(xpoints, c_t[1], label='line2')
line3, = ax.semilogy(xpoints, c_t[2], label='line3')
scatter = ax.plot(lambdas, error_probs, 'o')
ax.legend([line1, line2, line3], ["Steiner $(26,5,3)$, 3-IC code, $N=260$", "$(26,5)$ 2-IC code, $N=65780$", "(26,5) Random Code, $N=\infty$"])
plt.axhline(y = 1e-5, color = 'r', linestyle = '-')
_ = plt.xticks(lambdas) # Disable trash output by assigning it to _